<a href="https://colab.research.google.com/github/Auzek2002/IBA_DataThon_2.0_Round_2/blob/main/DataThon_2_0_Round_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Importing Libs:**

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import StackingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tensorflow.keras import datasets,layers,Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout,GlobalAveragePooling2D
from zipfile import ZipFile
import os,glob
from tqdm._tqdm_notebook import tqdm_notebook as tqdm
from keras.models import Model
from keras.layers import BatchNormalization
from zipfile import ZipFile
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from imblearn.over_sampling import SMOTE

<ipython-input-1-ec921842cf36>:25: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  from tqdm._tqdm_notebook import tqdm_notebook as tqdm


# **Loading the DataSet:**

In [ ]:
df = pd.read_excel('/content/train_data.xlsx')

In [ ]:
df.shape

(161393, 16)

In [ ]:
df.head()

,property_id,location_id,RowID,no_of_bathrooms,degree_of_latitude,degree_of_longitude,no_of_bedrooms,space_occupied,listing_purpose,category_property,listing_date,estate_agency,salesperson,area,has_garage,price
0,13845428,3847,30000,7.0,31.526751,74.337072,7.0,1 Kanal,For Rent,Houseo,2019-02-24,Raja Real Estate,Raja Adeeb,"Gulberg, Lahore, Punjab",False,300000
1,13851899,1487,30001,6.0,31.502385,74.414506,6.0,10 Marla,For Rent,House,2019-07-24,Dream Developers & Property Consultants,Ali Auwn,"Divine Gardens, Lahore, Punjab",False,90000
2,13852192,1447,30002,NaN,31.462493,74.409342,NaN,4 Marla,For Rent,Room,2018-12-26,Property Channel,Abbas Ali,"DHA Defence, Lahore, Punjab",True,15000
3,13855250,1006,30003,3.0,31.432105,74.349117,2.0,4 Marla,For Rent,Lower Portion,2019-06-24,Syed Associates & Builders,Syed Talha,"Pak Arab Housing Society, Lahore, Punjab",False,21000
4,13855556,1006,30004,6.0,31.432105,74.349117,4.0,5 Marla,For Rent,House,2019-07-18,Syed Associates & Builders,Syed Talha,"Pak Arab Housing Society, Lahore, Punjab",True,43000


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161393 entries, 0 to 161392
Data columns (total 16 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   property_id          161393 non-null  int64         
 1   location_id          161393 non-null  int64         
 2   RowID                161393 non-null  int64         
 3   no_of_bathrooms      129205 non-null  float64       
 4   degree_of_latitude   161393 non-null  float64       
 5   degree_of_longitude  161393 non-null  float64       
 6   no_of_bedrooms       145317 non-null  float64       
 7   space_occupied       161393 non-null  object        
 8   listing_purpose      161393 non-null  object        
 9   category_property    161393 non-null  object        
 10  listing_date         161393 non-null  datetime64[ns]
 11  estate_agency        134700 non-null  object        
 12  salesperson          134699 non-null  object        
 13  area          

In [ ]:
#What to do:
#1) Check for NULL values and remove them
#2) break Listing Date into diff features
#3) break the area column into three columns 'society', 'city' and 'province'
#4) Fix the space_occupied column
#5) Fix in correct entered text in any column
#6) Label Encoding and dropping columns

# **Step 1: Checking NULL values and remove them**

In [ ]:
df.isnull().sum()

property_id                0
location_id                0
RowID                      0
no_of_bathrooms        32188
degree_of_latitude         0
degree_of_longitude        0
no_of_bedrooms         16076
space_occupied             0
listing_purpose            0
category_property          0
listing_date               0
estate_agency          26693
salesperson            26694
area                       0
has_garage                 0
price                      0
dtype: int64

In [ ]:
df['estate_agency'].unique()

array(['Raja Real Estate', 'Dream Developers & Property Consultants',
       'Property Channel', ..., 'Al Raffay Estate', 'Khaira Estate',
       'DHS Housing Solutions'], dtype=object)

In [ ]:
df['estate_agency'].nunique()

5685

In [ ]:
df['salesperson'].unique()

array(['Raja Adeeb', 'Ali Auwn', 'Abbas Ali', ..., 'Zafraan Meer',
       'Zafran', 'TALHA   MIAN AHMAD'], dtype=object)

In [ ]:
df['salesperson'].nunique()

10647

# **Using KNN Imputer to fill missing values in bedroom and bathroom features:**

In [ ]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)  # You can adjust the number of neighbors
df[['no_of_bathrooms', 'no_of_bedrooms']] = imputer.fit_transform(df[['no_of_bathrooms', 'no_of_bedrooms']])

In [ ]:
# cols = df.columns
# for col in cols:
#   if df[col].dtype == 'O':
#     continue
#   else:
#     df[col].fillna(value=df[col].mean(),inplace=True)

In [ ]:
df.isnull().sum()

property_id                0
location_id                0
RowID                      0
no_of_bathrooms            0
degree_of_latitude         0
degree_of_longitude        0
no_of_bedrooms             0
space_occupied             0
listing_purpose            0
category_property          0
listing_date               0
estate_agency          26693
salesperson            26694
area                       0
has_garage                 0
price                      0
dtype: int64

# **Replacing NULL values in estate_agency and salesperson with the mode value(These features might be dropped later):**

In [ ]:
df['estate_agency'].fillna(value=df['estate_agency'].mode()[0],inplace=True)
df['salesperson'].fillna(value=df['salesperson'].mode()[0],inplace=True)

In [ ]:
df.isnull().sum()

property_id            0
location_id            0
RowID                  0
no_of_bathrooms        0
degree_of_latitude     0
degree_of_longitude    0
no_of_bedrooms         0
space_occupied         0
listing_purpose        0
category_property      0
listing_date           0
estate_agency          0
salesperson            0
area                   0
has_garage             0
price                  0
dtype: int64

# **Step 1 complete**

# **Step 2: break Listing Date into diff features**

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161393 entries, 0 to 161392
Data columns (total 16 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   property_id          161393 non-null  int64         
 1   location_id          161393 non-null  int64         
 2   RowID                161393 non-null  int64         
 3   no_of_bathrooms      161393 non-null  float64       
 4   degree_of_latitude   161393 non-null  float64       
 5   degree_of_longitude  161393 non-null  float64       
 6   no_of_bedrooms       161393 non-null  float64       
 7   space_occupied       161393 non-null  object        
 8   listing_purpose      161393 non-null  object        
 9   category_property    161393 non-null  object        
 10  listing_date         161393 non-null  datetime64[ns]
 11  estate_agency        161393 non-null  object        
 12  salesperson          161393 non-null  object        
 13  area          

In [ ]:
df['listing_year'] = df['listing_date'].dt.year
df['listing_month'] = df['listing_date'].dt.month
df['listing_day'] = df['listing_date'].dt.day

# Drop the original 'listing_date' column
df = df.drop('listing_date', axis=1)

In [ ]:
df.head()

,property_id,location_id,RowID,no_of_bathrooms,degree_of_latitude,degree_of_longitude,no_of_bedrooms,space_occupied,listing_purpose,category_property,estate_agency,salesperson,area,has_garage,price,listing_year,listing_month,listing_day
0,13845428,3847,30000,7.000000,31.526751,74.337072,7.000000,1 Kanal,For Rent,Houseo,Raja Real Estate,Raja Adeeb,"Gulberg, Lahore, Punjab",False,300000,2019,2,24
1,13851899,1487,30001,6.000000,31.502385,74.414506,6.000000,10 Marla,For Rent,House,Dream Developers & Property Consultants,Ali Auwn,"Divine Gardens, Lahore, Punjab",False,90000,2019,7,24
2,13852192,1447,30002,4.350877,31.462493,74.409342,4.511881,4 Marla,For Rent,Room,Property Channel,Abbas Ali,"DHA Defence, Lahore, Punjab",True,15000,2018,12,26
3,13855250,1006,30003,3.000000,31.432105,74.349117,2.000000,4 Marla,For Rent,Lower Portion,Syed Associates & Builders,Syed Talha,"Pak Arab Housing Society, Lahore, Punjab",False,21000,2019,6,24
4,13855556,1006,30004,6.000000,31.432105,74.349117,4.000000,5 Marla,For Rent,House,Syed Associates & Builders,Syed Talha,"Pak Arab Housing Society, Lahore, Punjab",True,43000,2019,7,18


# **Step 2 complete**

# **Step 3: break the area column into three columns 'society', 'city' and 'province'**

In [ ]:
df_1 = df
split_df = df_1['area'].str.split(', ', expand=True)

# Assign the split columns to the DataFrame
df_1['society'] = split_df[0]
df_1['city'] = split_df[1]
df_1['province'] = split_df[2]

# Drop the original 'area' column if needed
df_1 = df_1.drop('area', axis=1)

# **Step 3 complete**

# **Step 4: Fix the space_occupied column**

In [ ]:
df_1.head()

,property_id,location_id,RowID,no_of_bathrooms,degree_of_latitude,degree_of_longitude,no_of_bedrooms,space_occupied,listing_purpose,category_property,estate_agency,salesperson,has_garage,price,listing_year,listing_month,listing_day,society,city,province
0,13845428,3847,30000,7.000000,31.526751,74.337072,7.000000,1 Kanal,For Rent,Houseo,Raja Real Estate,Raja Adeeb,False,300000,2019,2,24,Gulberg,Lahore,Punjab
1,13851899,1487,30001,6.000000,31.502385,74.414506,6.000000,10 Marla,For Rent,House,Dream Developers & Property Consultants,Ali Auwn,False,90000,2019,7,24,Divine Gardens,Lahore,Punjab
2,13852192,1447,30002,4.350877,31.462493,74.409342,4.511881,4 Marla,For Rent,Room,Property Channel,Abbas Ali,True,15000,2018,12,26,DHA Defence,Lahore,Punjab
3,13855250,1006,30003,3.000000,31.432105,74.349117,2.000000,4 Marla,For Rent,Lower Portion,Syed Associates & Builders,Syed Talha,False,21000,2019,6,24,Pak Arab Housing Society,Lahore,Punjab
4,13855556,1006,30004,6.000000,31.432105,74.349117,4.000000,5 Marla,For Rent,House,Syed Associates & Builders,Syed Talha,True,43000,2019,7,18,Pak Arab Housing Society,Lahore,Punjab


In [ ]:
# Define conversion factors
kanal_to_sqft = 5445  # 1 Kanal is approximately 5445 square feet
marla_to_sqft = 272.25  # 1 Marla is 1/20th of a Kanal

# Replace commas and convert 'space_occupied' to float
def convert_space_to_float(space):
    try:
        return float(space.replace(',', ''))
    except ValueError:
        return None

# Convert 'space_occupied' to square feet
def convert_space_to_sqft(space):
    try:
        value = float(space.split()[0])  # Extract the numeric value
        unit = space.split()[1]  # Extract the unit (e.g., 'Kanal', 'Marla')
        if value <= 0:  # Check if the value is 0 or negative
            return round(default_value * kanal_to_sqft) if 'Kanal' in space else round(default_value * marla_to_sqft)
        if unit == 'Kanal':
            return round(value * kanal_to_sqft)
        elif unit == 'Marla':
            return round(value * marla_to_sqft)
        else:
            return None  # Handle unknown units
    except (ValueError, IndexError):
        return None  # Handle missing or invalid data

# Apply conversion to 'space_occupied' to get non-zero values
non_zero_values = df_1[df_1['space_occupied'] != '0']['space_occupied'].apply(convert_space_to_float)

# Calculate default value as the mean of non-zero values
default_value = non_zero_values.mean()

# Apply conversion to 'space_occupied' using the default value
df_1['space_occupied'] = df_1['space_occupied'].apply(convert_space_to_sqft)

In [ ]:
df_1.head()

,property_id,location_id,RowID,no_of_bathrooms,degree_of_latitude,degree_of_longitude,no_of_bedrooms,space_occupied,listing_purpose,category_property,estate_agency,salesperson,has_garage,price,listing_year,listing_month,listing_day,society,city,province
0,13845428,3847,30000,7.000000,31.526751,74.337072,7.000000,5445.0,For Rent,Houseo,Raja Real Estate,Raja Adeeb,False,300000,2019,2,24,Gulberg,Lahore,Punjab
1,13851899,1487,30001,6.000000,31.502385,74.414506,6.000000,2722.0,For Rent,House,Dream Developers & Property Consultants,Ali Auwn,False,90000,2019,7,24,Divine Gardens,Lahore,Punjab
2,13852192,1447,30002,4.350877,31.462493,74.409342,4.511881,1089.0,For Rent,Room,Property Channel,Abbas Ali,True,15000,2018,12,26,DHA Defence,Lahore,Punjab
3,13855250,1006,30003,3.000000,31.432105,74.349117,2.000000,1089.0,For Rent,Lower Portion,Syed Associates & Builders,Syed Talha,False,21000,2019,6,24,Pak Arab Housing Society,Lahore,Punjab
4,13855556,1006,30004,6.000000,31.432105,74.349117,4.000000,1361.0,For Rent,House,Syed Associates & Builders,Syed Talha,True,43000,2019,7,18,Pak Arab Housing Society,Lahore,Punjab


In [ ]:
df_1['space_occupied'].fillna(value=df_1['space_occupied'].mean(),inplace=True)

In [ ]:
df_1.isnull().sum()

property_id            0
location_id            0
RowID                  0
no_of_bathrooms        0
degree_of_latitude     0
degree_of_longitude    0
no_of_bedrooms         0
space_occupied         0
listing_purpose        0
category_property      0
estate_agency          0
salesperson            0
has_garage             0
price                  0
listing_year           0
listing_month          0
listing_day            0
society                0
city                   0
province               0
dtype: int64

# **Step 4 complete**

# **Step 5: Fix in correct entered text in any column**

In [ ]:
df_1.head()

,property_id,location_id,RowID,no_of_bathrooms,degree_of_latitude,degree_of_longitude,no_of_bedrooms,space_occupied,listing_purpose,category_property,estate_agency,salesperson,has_garage,price,listing_year,listing_month,listing_day,society,city,province
0,13845428,3847,30000,7.000000,31.526751,74.337072,7.000000,5445.0,For Rent,Houseo,Raja Real Estate,Raja Adeeb,False,300000,2019,2,24,Gulberg,Lahore,Punjab
1,13851899,1487,30001,6.000000,31.502385,74.414506,6.000000,2722.0,For Rent,House,Dream Developers & Property Consultants,Ali Auwn,False,90000,2019,7,24,Divine Gardens,Lahore,Punjab
2,13852192,1447,30002,4.350877,31.462493,74.409342,4.511881,1089.0,For Rent,Room,Property Channel,Abbas Ali,True,15000,2018,12,26,DHA Defence,Lahore,Punjab
3,13855250,1006,30003,3.000000,31.432105,74.349117,2.000000,1089.0,For Rent,Lower Portion,Syed Associates & Builders,Syed Talha,False,21000,2019,6,24,Pak Arab Housing Society,Lahore,Punjab
4,13855556,1006,30004,6.000000,31.432105,74.349117,4.000000,1361.0,For Rent,House,Syed Associates & Builders,Syed Talha,True,43000,2019,7,18,Pak Arab Housing Society,Lahore,Punjab


In [ ]:
df_1['listing_purpose'].unique() #OK

array(['For Rent', 'For Sale'], dtype=object)

In [ ]:
df_1['category_property'].unique()

array(['Houseo', 'House', 'Room', 'Lower Portion', 'Upper Portion',
       'Flat', 'Lower Portions', 'Housei', 'Housew', 'Flatw', 'Roomp',
       'Flatv', 'Roomw', 'Flatj', 'Upper Portionb', 'Flaty', 'Flatg',
       'Housej', 'Lower Portionw', 'Flatb', 'Upper Portionv', 'Houseg',
       'Housel', 'Flati', 'Upper Portionh', 'Rooma', 'Lower Portionu',
       'Houser', 'Housef', 'Lower Portionm', 'Farm House', 'Flatz',
       'Housev', 'Lower Portionq', 'Lower Portionn', 'Houses', 'Housec',
       'Housea', 'Housee', 'Housez', 'Houset', 'Houseu', 'Housen',
       'Housep', 'Househ', 'Housey', 'Penthouse', 'Flate', 'Flatn',
       'Houseb', 'Flatr', 'Housex', 'Upper Portiong', 'Upper Portione',
       'Flats', 'Flatp', 'Roomv', 'Upper Portionf', 'Lower Portiong',
       'Upper Portionx', 'Penthousen', 'Flatf', 'Upper Portionr',
       'Lower Portiond', 'Upper Portiont', 'Lower Portiont',
       'Upper Portioni', 'Housek', 'Upper Portiona', 'Housem', 'Flatu',
       'Flatd', 'Housed', 'Flat

In [ ]:
import re

# Define mapping for different patterns
pattern_mapping = {
    r'^House[a-z]*$': 'House',
    r'^Flat[a-z]*$': 'Flat',
    r'^Room[a-z]*$': 'Room',
    r'^Lower Portion[a-z]*$': 'Lower Portion',
    r'^Upper Portion[a-z]*$': 'Upper Portion',
    r'^Farm House[a-z]*$': 'Farm House',
    r'^Penthouse[a-z]*$': 'Penthouse'
}

# Apply the mappings using replace() with regex
for pattern, replacement in pattern_mapping.items():
    df_1['category_property'] = df_1['category_property'].str.replace(pattern, replacement, regex=True)

In [ ]:
df_1['category_property'].unique() #OK

array(['House', 'Room', 'Lower Portion', 'Upper Portion', 'Flat',
       'Farm House', 'Penthouse'], dtype=object)

In [ ]:
df_1['city'] = df_1['city'].replace({'Islamabad Capital': 'Islamabad','Punjab':'Islamabad'})

In [ ]:
df_1['city'] = df_1['city'].replace({'Sindh':'Karachi'})
df_1['city'].unique() #OK

array(['Lahore', 'Karachi', 'Islamabad', 'Faisalabad', 'Rawalpindi'],
      dtype=object)

In [ ]:
df_1['province'].unique()

array(['Punjab', 'Sindh', 'Islamabad Capital', 'Faisalabad', 'Lahore',
       'Islamabad', 'Karachi', 'Rawalpindi'], dtype=object)

In [ ]:
df_1['province'] = df_1['province'].replace({'Islamabad': 'Punjab','Islamabad Capital': 'Punjab','Karachi': 'Sindh', 'Rawalpindi':'Punjab', 'Faisalabad':'Punjab', 'Lahore':'Punjab'})

In [ ]:
df_1['province'].unique() #OK

array(['Punjab', 'Sindh'], dtype=object)

In [ ]:
df_1.head()

,property_id,location_id,RowID,no_of_bathrooms,degree_of_latitude,degree_of_longitude,no_of_bedrooms,space_occupied,listing_purpose,category_property,estate_agency,salesperson,has_garage,price,listing_year,listing_month,listing_day,society,city,province
0,13845428,3847,30000,7.000000,31.526751,74.337072,7.000000,5445.0,For Rent,House,Raja Real Estate,Raja Adeeb,False,300000,2019,2,24,Gulberg,Lahore,Punjab
1,13851899,1487,30001,6.000000,31.502385,74.414506,6.000000,2722.0,For Rent,House,Dream Developers & Property Consultants,Ali Auwn,False,90000,2019,7,24,Divine Gardens,Lahore,Punjab
2,13852192,1447,30002,4.350877,31.462493,74.409342,4.511881,1089.0,For Rent,Room,Property Channel,Abbas Ali,True,15000,2018,12,26,DHA Defence,Lahore,Punjab
3,13855250,1006,30003,3.000000,31.432105,74.349117,2.000000,1089.0,For Rent,Lower Portion,Syed Associates & Builders,Syed Talha,False,21000,2019,6,24,Pak Arab Housing Society,Lahore,Punjab
4,13855556,1006,30004,6.000000,31.432105,74.349117,4.000000,1361.0,For Rent,House,Syed Associates & Builders,Syed Talha,True,43000,2019,7,18,Pak Arab Housing Society,Lahore,Punjab


# **Step 5 complete**

# **Dropping Columns : 'property_id', 'location_id', 'RowID', 'estate_agency', 'salesperson'**

In [ ]:
columns_to_drop = ['property_id', 'location_id', 'RowID', 'estate_agency', 'salesperson']
df_1 = df_1.drop(columns_to_drop, axis=1)

In [ ]:
df_1.head()

,no_of_bathrooms,degree_of_latitude,degree_of_longitude,no_of_bedrooms,space_occupied,listing_purpose,category_property,has_garage,price,listing_year,listing_month,listing_day,society,city,province
0,7.000000,31.526751,74.337072,7.000000,5445.0,For Rent,House,False,300000,2019,2,24,Gulberg,Lahore,Punjab
1,6.000000,31.502385,74.414506,6.000000,2722.0,For Rent,House,False,90000,2019,7,24,Divine Gardens,Lahore,Punjab
2,4.350877,31.462493,74.409342,4.511881,1089.0,For Rent,Room,True,15000,2018,12,26,DHA Defence,Lahore,Punjab
3,3.000000,31.432105,74.349117,2.000000,1089.0,For Rent,Lower Portion,False,21000,2019,6,24,Pak Arab Housing Society,Lahore,Punjab
4,6.000000,31.432105,74.349117,4.000000,1361.0,For Rent,House,True,43000,2019,7,18,Pak Arab Housing Society,Lahore,Punjab


In [ ]:
# from sklearn.decomposition import PCA
# from sklearn.preprocessing import StandardScaler
# import pandas as pd

# # Assume 'df_1' is your dataframe with one-hot encoded categorical variables and numerical features
# # Concatenate the one-hot encoded features with numerical features
# numerical_features = ['no_of_bathrooms', 'degree_of_latitude', 'degree_of_longitude', 'no_of_bedrooms', 'space_occupied', 'has_garage','listing_year','listing_month','listing_day']
# categorical_features = ['listing_purpose', 'category_property', 'society', 'city', 'province']

# # Apply one-hot encoding to categorical features
# one_hot_encoded_features = pd.get_dummies(df_1[categorical_features])

# # Concatenate one-hot encoded features with numerical features
# all_features = pd.concat([df_1[numerical_features], one_hot_encoded_features], axis=1)

# # Standardize the data
# scaler = StandardScaler()
# scaled_data = scaler.fit_transform(all_features)

# # Apply PCA
# pca = PCA(n_components=0.95)  # Choose number of components to explain 95% of variance
# pca_result = pca.fit_transform(scaled_data)

# # Convert PCA result to DataFrame
# pca_df_1 = pd.DataFrame(data=pca_result, columns=[f"PC{i+1}" for i in range(pca_result.shape[1])])

# # Concatenate PCA result with original DataFrame
# final_df_1 = pd.concat([df_1[['price', 'listing_year', 'listing_month', 'listing_day']], pca_df_1], axis=1)
# from sklearn.decomposition import PCA
# from sklearn.preprocessing import StandardScaler
# import pandas as pd

# # Assuming df_1 is your dataframe

# # Concatenate the one-hot encoded features with numerical features
# numerical_features = ['no_of_bathrooms', 'degree_of_latitude', 'degree_of_longitude', 'no_of_bedrooms', 'space_occupied', 'has_garage','listing_year','listing_month','listing_day']
# categorical_features = ['listing_purpose', 'category_property', 'society', 'city', 'province']

# # Apply one-hot encoding to categorical features
# one_hot_encoded_features = pd.get_dummies(df_1[categorical_features])

# # Concatenate one-hot encoded features with numerical features
# all_features = pd.concat([df_1[numerical_features], one_hot_encoded_features], axis=1)

# # Standardize the data
# scaler = StandardScaler()
# scaled_data = scaler.fit_transform(all_features)

# # Apply PCA with a fixed number of components
# pca = PCA(n_components=100)
# pca_result = pca.fit_transform(scaled_data)

# # Convert PCA result to DataFrame
# pca_df_1 = pd.DataFrame(data=pca_result, columns=[f"PC{i+1}" for i in range(pca_result.shape[1])])

# # Concatenate PCA result with original DataFrame
# final_df_1 = pd.concat([df_1[['price', 'listing_year', 'listing_month', 'listing_day']], pca_df_1], axis=1)

# # Display final_df_1
# final_df_1
#Didn't use due to runtime(Got disconnected) issue


# **Modeling the data:**

In [ ]:
# x = final_df_1.drop(['price','listing_year'],axis=1)
# y = final_df_1['price']
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
# x_train.shape

In [ ]:
df.head()

,property_id,location_id,RowID,no_of_bathrooms,degree_of_latitude,degree_of_longitude,no_of_bedrooms,space_occupied,listing_purpose,category_property,...,salesperson,area,has_garage,price,listing_year,listing_month,listing_day,society,city,province
0,13845428,3847,30000,7.000000,31.526751,74.337072,7.000000,1 Kanal,For Rent,Houseo,...,Raja Adeeb,"Gulberg, Lahore, Punjab",False,300000,2019,2,24,Gulberg,Lahore,Punjab
1,13851899,1487,30001,6.000000,31.502385,74.414506,6.000000,10 Marla,For Rent,House,...,Ali Auwn,"Divine Gardens, Lahore, Punjab",False,90000,2019,7,24,Divine Gardens,Lahore,Punjab
2,13852192,1447,30002,4.350877,31.462493,74.409342,4.511881,4 Marla,For Rent,Room,...,Abbas Ali,"DHA Defence, Lahore, Punjab",True,15000,2018,12,26,DHA Defence,Lahore,Punjab
3,13855250,1006,30003,3.000000,31.432105,74.349117,2.000000,4 Marla,For Rent,Lower Portion,...,Syed Talha,"Pak Arab Housing Society, Lahore, Punjab",False,21000,2019,6,24,Pak Arab Housing Society,Lahore,Punjab
4,13855556,1006,30004,6.000000,31.432105,74.349117,4.000000,5 Marla,For Rent,House,...,Syed Talha,"Pak Arab Housing Society, Lahore, Punjab",True,43000,2019,7,18,Pak Arab Housing Society,Lahore,Punjab


In [ ]:
split_df = df['area'].str.split(', ', expand=True)

# Assign the split columns to the DataFrame
df['society'] = split_df[0]
df['city'] = split_df[1]
df['province'] = split_df[2]

# Drop the original 'area' column if needed
df = df.drop('area', axis=1)

In [ ]:
kanal_to_sqft = 5445  # 1 Kanal is approximately 5445 square feet
marla_to_sqft = 272.25  # 1 Marla is 1/20th of a Kanal

# Replace commas and convert 'space_occupied' to float
def convert_space_to_float(space):
    try:
        return float(space.replace(',', ''))
    except ValueError:
        return None

# Convert 'space_occupied' to square feet
def convert_space_to_sqft(space):
    try:
        value = float(space.split()[0])  # Extract the numeric value
        unit = space.split()[1]  # Extract the unit (e.g., 'Kanal', 'Marla')
        if value <= 0:  # Check if the value is 0 or negative
            return round(default_value * kanal_to_sqft) if 'Kanal' in space else round(default_value * marla_to_sqft)
        if unit == 'Kanal':
            return round(value * kanal_to_sqft)
        elif unit == 'Marla':
            return round(value * marla_to_sqft)
        else:
            return None  # Handle unknown units
    except (ValueError, IndexError):
        return None  # Handle missing or invalid data

# Apply conversion to 'space_occupied' to get non-zero values
non_zero_values = df[df['space_occupied'] != '0']['space_occupied'].apply(convert_space_to_float)

# Calculate default value as the mean of non-zero values
default_value = non_zero_values.mean()

# Apply conversion to 'space_occupied' using the default value
df['space_occupied'] = df['space_occupied'].apply(convert_space_to_sqft)

In [ ]:
df['space_occupied'].fillna(value=df['space_occupied'].mean(),inplace=True)

In [ ]:
import re

# Define mapping for different patterns
pattern_mapping = {
    r'^House[a-z]*$': 'House',
    r'^Flat[a-z]*$': 'Flat',
    r'^Room[a-z]*$': 'Room',
    r'^Lower Portion[a-z]*$': 'Lower Portion',
    r'^Upper Portion[a-z]*$': 'Upper Portion',
    r'^Farm House[a-z]*$': 'Farm House',
    r'^Penthouse[a-z]*$': 'Penthouse'
}

# Apply the mappings using replace() with regex
for pattern, replacement in pattern_mapping.items():
    df['category_property'] = df['category_property'].str.replace(pattern, replacement, regex=True)

In [ ]:
df['city'] = df['city'].replace({'Islamabad Capital': 'Islamabad','Punjab':'Islamabad'})

In [ ]:
df['city'] = df['city'].replace({'Sindh':'Karachi'})
df['city'].unique() #OK

array(['Lahore', 'Karachi', 'Islamabad', 'Faisalabad', 'Rawalpindi'],
      dtype=object)

In [ ]:
df['province'] = df['province'].replace({'Islamabad': 'Punjab','Islamabad Capital': 'Punjab','Karachi': 'Sindh', 'Rawalpindi':'Punjab', 'Faisalabad':'Punjab', 'Lahore':'Punjab'})

In [ ]:
columns_to_drop = ['property_id', 'location_id', 'RowID', 'estate_agency', 'salesperson']
df = df.drop(columns_to_drop, axis=1)

In [ ]:
df.head()

,no_of_bathrooms,degree_of_latitude,degree_of_longitude,no_of_bedrooms,space_occupied,listing_purpose,category_property,has_garage,price,listing_year,listing_month,listing_day,society,city,province
0,7.000000,31.526751,74.337072,7.000000,5445.0,For Rent,House,False,300000,2019,2,24,Gulberg,Lahore,Punjab
1,6.000000,31.502385,74.414506,6.000000,2722.0,For Rent,House,False,90000,2019,7,24,Divine Gardens,Lahore,Punjab
2,4.350877,31.462493,74.409342,4.511881,1089.0,For Rent,Room,True,15000,2018,12,26,DHA Defence,Lahore,Punjab
3,3.000000,31.432105,74.349117,2.000000,1089.0,For Rent,Lower Portion,False,21000,2019,6,24,Pak Arab Housing Society,Lahore,Punjab
4,6.000000,31.432105,74.349117,4.000000,1361.0,For Rent,House,True,43000,2019,7,18,Pak Arab Housing Society,Lahore,Punjab


In [ ]:
df_2 = df.drop(['society','listing_year'],axis=1)

In [ ]:
encoder = LabelEncoder()

In [ ]:
col = df_2.columns

In [ ]:
for i in col:
  if df_2[i].dtype == 'O':
    encoder.fit(df_2[i])
    df_2[i] = encoder.transform(df_2[i])

In [ ]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161393 entries, 0 to 161392
Data columns (total 13 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   no_of_bathrooms      161393 non-null  float64
 1   degree_of_latitude   161393 non-null  float64
 2   degree_of_longitude  161393 non-null  float64
 3   no_of_bedrooms       161393 non-null  float64
 4   space_occupied       161393 non-null  float64
 5   listing_purpose      161393 non-null  int64  
 6   category_property    161393 non-null  int64  
 7   has_garage           161393 non-null  bool   
 8   price                161393 non-null  int64  
 9   listing_month        161393 non-null  int32  
 10  listing_day          161393 non-null  int32  
 11  city                 161393 non-null  int64  
 12  province             161393 non-null  int64  
dtypes: bool(1), float64(5), int32(2), int64(5)
memory usage: 13.7 MB


In [ ]:
df_2.head()

,no_of_bathrooms,degree_of_latitude,degree_of_longitude,no_of_bedrooms,space_occupied,listing_purpose,category_property,has_garage,price,listing_month,listing_day,city,province
0,7.000000,31.526751,74.337072,7.000000,5445.0,0,2,False,300000,2,24,3,0
1,6.000000,31.502385,74.414506,6.000000,2722.0,0,2,False,90000,7,24,3,0
2,4.350877,31.462493,74.409342,4.511881,1089.0,0,5,True,15000,12,26,3,0
3,3.000000,31.432105,74.349117,2.000000,1089.0,0,3,False,21000,6,24,3,0
4,6.000000,31.432105,74.349117,4.000000,1361.0,0,2,True,43000,7,18,3,0


In [ ]:
x = df_2.drop(['price'],axis=1)
y = df_2['price']
x_train , x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# **Scaling the data:**

In [ ]:
scaler = StandardScaler()

In [ ]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

# **Linear Regression**

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Train a linear regression model
model = LinearRegression()
model.fit(x_train, y_train)

# Make predictions
y = model.predict(x_train)
y_pred = model.predict(x_test)

# Evaluate the model
mse = mean_squared_error(y_train, y)
mse_t = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error Training : {mse}")
print(f"Mean Squared Error: {mse_t}")
# Mean Squared Error Training : 897249192413764.2
# Mean Squared Error: 866451758756395.1

Mean Squared Error Training : 882026878761007.5
Mean Squared Error: 865898876876363.9


# **Neural Networks:**

In [ ]:
x_train.shape

(129114, 12)

In [ ]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 1.7 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import Hyperband
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd

# Assume X_train, y_train, X_test, y_test are your training and test data

def build_model(hp):
    model = keras.Sequential()
    model.add(layers.InputLayer(input_shape=(x_train.shape[1],)))
    for i in range(hp.Int('num_layers', 1, 5)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i), 32, 512, 32), activation='relu'))
        model.add(layers.Dropout(hp.Float('dropout_' + str(i), 0, 0.5, step=0.1, default=0.2)))
    model.add(layers.Dense(1, activation='linear'))
    model.compile(optimizer='adam', loss='mse')
    return model

tuner = Hyperband(
    build_model,
    objective='val_loss',
    max_epochs=20,
    hyperband_iterations=2,
    overwrite=True,
    directory='keras_tuner_logs',
    project_name='my_house_price_prediction'
)

In [ ]:
# Split the data
X_train, X_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

In [ ]:
tuner.search(X_train, y_train, epochs=30, validation_data=(X_val, y_val))

In [ ]:
best_model = tuner.get_best_models(num_models=1)[0]
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]

In [ ]:
best_model.fit(X_train, y_train, epochs=5, validation_data=(X_val, y_val))

Epoch 1/5
3228/3228 [==============================] - 63s 19ms/step - loss: 270231993843712.0000 - val_loss: 328052554858496.0000
Epoch 2/5
3228/3228 [==============================] - 52s 16ms/step - loss: 305243829043200.0000 - val_loss: 352614130647040.0000
Epoch 3/5
3228/3228 [==============================] - 65s 20ms/step - loss: 285960600289280.0000 - val_loss: 353197407338496.0000
Epoch 4/5
3228/3228 [==============================] - 44s 14ms/step - loss: 294228512997376.0000 - val_loss: 340367199174656.0000
Epoch 5/5
3228/3228 [==============================] - 47s 15ms/step - loss: 279196312010752.0000 - val_loss: 389222385057792.0000


In [ ]:
y_pred = best_model.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
print("Test MSE:", mse)

In [ ]:
# model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
# model.summary()

In [ ]:
# early_stopping = EarlyStopping(patience=5, restore_best_weights=True)
# history = model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test), callbacks=[early_stopping])

In [ ]:
# mse = model.evaluate(x_test, y_test)
# print(f"Mean Squared Error on Test Set: {mse}")
# # loss: 770021668159488.0000
# # Mean Squared Error on Test Set: 770021668159488.0

# **XGBoost**

In [ ]:
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# XGBoost
xgb_reg = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, seed=42)
xgb_reg.fit(x_train, y_train)
y_pred_xgb_train = xgb_reg.predict(x_train)
y_pred_xgb_test = xgb_reg.predict(x_test)
mse_xgb_train = mean_squared_error(y_train, y_pred_xgb_train)
mse_xgb_test = mean_squared_error(y_test, y_pred_xgb_test)
print(f"XGBoost Train MSE: {mse_xgb_train}, Test MSE: {mse_xgb_test}")

In [ ]:
price_column = df_2.pop('price')  # Extract 'price' column
df_2['price'] = price_column

In [ ]:
df_2.to_csv('Transformed_Data_G2.csv')

# **Random Forest**

In [ ]:
rf_reg = RandomForestRegressor(n_estimators=100, random_state=42)
rf_reg.fit(x_train, y_train)
y_pred_rf_train = rf_reg.predict(x_train)
y_pred_rf_test = rf_reg.predict(x_test)
mse_rf_train = mean_squared_error(y_train, y_pred_rf_train)
mse_rf_test = mean_squared_error(y_test, y_pred_rf_test)
print(f"Random Forest Train MSE: {mse_rf_train}, Test MSE: {mse_rf_test}")

# **Lasso Regression**

In [ ]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge
lasso = Lasso(alpha=0.1)
lasso.fit(x_train, y_train)
y_pred_lasso_train = lasso.predict(x_train)
y_pred_lasso_test = lasso.predict(x_test)
mse_lasso_train = mean_squared_error(y_train, y_pred_lasso_train)
mse_lasso_test = mean_squared_error(y_test, y_pred_lasso_test)
print(f"Lasso Regression Train MSE: {mse_lasso_train}, Test MSE: {mse_lasso_test}")

# **Ridge Regression**

In [ ]:
ridge = Ridge(alpha=0.1)
ridge.fit(x_train, y_train)
y_pred_ridge_train = ridge.predict(x_train)
y_pred_ridge_test = ridge.predict(x_test)
mse_ridge_train = mean_squared_error(y_train, y_pred_ridge_train)
mse_ridge_test = mean_squared_error(y_test, y_pred_ridge_test)
print(f"Ridge Regression Train MSE: {mse_ridge_train}, Test MSE: {mse_ridge_test}")

# **SVR**

In [ ]:
from sklearn.svm import SVR
svr = SVR(kernel='linear')
svr.fit(x_train, y_train)
y_pred_svr_train = svr.predict(x_train)
y_pred_svr_test = svr.predict(x_test)
mse_svr_train = mean_squared_error(y_train, y_pred_svr_train)
mse_svr_test = mean_squared_error(y_test, y_pred_svr_test)
print(f"SVR Train MSE: {mse_svr_train}, Test MSE: {mse_svr_test}")

# **Decision Trees**

In [ ]:
from sklearn.tree import DecisionTreeRegressor
dt = DecisionTreeRegressor()
dt.fit(x_train, y_train)
y_pred_dt_train = dt.predict(x_train)
y_pred_dt_test = dt.predict(x_test)
mse_dt_train = mean_squared_error(y_train, y_pred_dt_train)
mse_dt_test = mean_squared_error(y_test, y_pred_dt_test)
print(f"Decision Tree Train MSE: {mse_dt_train}, Test MSE: {mse_dt_test}")

# **Making Predictions using Neural Nets:**

In [ ]:
df_test = pd.read_excel('/content/test_data.xlsx')

In [ ]:
df_test.isnull().sum()

property_id                0
location_id                0
RowID                      0
no_of_bathrooms         6091
degree_of_latitude         0
degree_of_longitude        0
no_of_bedrooms          3063
space_occupied             0
listing_purpose            0
category_property          0
listing_date               0
estate_agency          20686
salesperson            20686
area                       0
has_garage                 0
dtype: int64

In [ ]:
cols = df_test.columns
for col in cols:
  if df_test[col].dtype == 'O':
    continue
  else:
    df_test[col].fillna(value=df_test[col].mean(),inplace=True)

In [ ]:
df_test['estate_agency'].fillna(value=df_test['estate_agency'].mode()[0],inplace=True)
df_test['salesperson'].fillna(value=df_test['salesperson'].mode()[0],inplace=True)

In [ ]:
df_test['listing_year'] = df_test['listing_date'].dt.year
df_test['listing_month'] = df_test['listing_date'].dt.month
df_test['listing_day'] = df_test['listing_date'].dt.day

# Drop the original 'listing_date' column
df_test = df_test.drop('listing_date', axis=1)

In [ ]:
# Define conversion factors
kanal_to_sqft = 5445  # 1 Kanal is approximately 5445 square feet
marla_to_sqft = 272.25  # 1 Marla is 1/20th of a Kanal

# Replace commas and convert 'space_occupied' to float
def convert_space_to_float(space):
    try:
        return float(space.replace(',', ''))
    except ValueError:
        return None

# Convert 'space_occupied' to square feet
def convert_space_to_sqft(space):
    try:
        value = float(space.split()[0])  # Extract the numeric value
        unit = space.split()[1]  # Extract the unit (e.g., 'Kanal', 'Marla')
        if value <= 0:  # Check if the value is 0 or negative
            return round(default_value * kanal_to_sqft) if 'Kanal' in space else round(default_value * marla_to_sqft)
        if unit == 'Kanal':
            return round(value * kanal_to_sqft)
        elif unit == 'Marla':
            return round(value * marla_to_sqft)
        else:
            return None  # Handle unknown units
    except (ValueError, IndexError):
        return None  # Handle missing or invalid data

# Apply conversion to 'space_occupied' to get non-zero values
non_zero_values = df_test[df_test['space_occupied'] != '0']['space_occupied'].apply(convert_space_to_float)

# Calculate default value as the mean of non-zero values
default_value = non_zero_values.mean()

# Apply conversion to 'space_occupied' using the default value
df_test['space_occupied'] = df_test['space_occupied'].apply(convert_space_to_sqft)

In [ ]:
df['space_occupied'].fillna(value=df['space_occupied'].mean(),inplace=True)

In [ ]:
import re

# Define mapping for different patterns
pattern_mapping = {
    r'^House[a-z]*$': 'House',
    r'^Flat[a-z]*$': 'Flat',
    r'^Room[a-z]*$': 'Room',
    r'^Lower Portion[a-z]*$': 'Lower Portion',
    r'^Upper Portion[a-z]*$': 'Upper Portion',
    r'^Farm House[a-z]*$': 'Farm House',
    r'^Penthouse[a-z]*$': 'Penthouse'
}

# Apply the mappings using replace() with regex
for pattern, replacement in pattern_mapping.items():
    df_test['category_property'] = df_test['category_property'].str.replace(pattern, replacement, regex=True)

In [ ]:

split_df = df_test['area'].str.split(', ', expand=True)

# Assign the split columns to the DataFrame
df_test['society'] = split_df[0]
df_test['city'] = split_df[1]
df_test['province'] = split_df[2]

# Drop the original 'area' column if needed
df_test = df_test.drop('area', axis=1)

In [ ]:
df_test['city'] = df_test['city'].replace({'Islamabad Capital': 'Islamabad','Punjab':'Islamabad'})

In [ ]:
df_test['city'] = df_test['city'].replace({'Sindh':'Karachi'})
df_test['city'].unique() #OK

array(['Lahore', 'Karachi', 'Islamabad', 'Faisalabad', 'Rawalpindi'],
      dtype=object)

In [ ]:
df_test['province'] = df_test['province'].replace({'Islamabad': 'Punjab','Islamabad Capital': 'Punjab','Karachi': 'Sindh', 'Rawalpindi':'Punjab', 'Faisalabad':'Punjab', 'Lahore':'Punjab'})

In [ ]:
columns_to_drop = ['property_id', 'location_id', 'RowID', 'estate_agency', 'salesperson']
df_test = df_test.drop(columns_to_drop, axis=1)

In [ ]:
df_2 = df_test.drop(['society','listing_year'],axis=1)

In [ ]:
encoder = LabelEncoder()
col = df_2.columns

In [ ]:
for i in col:
  if df_2[i].dtype == 'O':
    encoder.fit(df_2[i])
    df_2[i] = encoder.transform(df_2[i])

In [ ]:
scaler = StandardScaler()
x = df_2

In [ ]:
scaler = StandardScaler()
x = scaler.fit_transform(x)

In [ ]:
y_pred = best_model.predict(x)

938/938 [==============================] - 2s 2ms/step


In [ ]:
y_pred_final_nn_binary = (y_pred).astype(int)

In [ ]:
y_pred_final_nn_binary

array([[510921792],
       [ 38030612],
       [ 10237187],
       ...,
       [   269534],
       [     1496],
       [    21610]])

In [ ]:
y_pred_final_nn_binary = (y_pred.flatten()).astype(int)

# Create DataFrame
new_df = pd.DataFrame({'rowID': range(1, len(df_2)+1), 'price': y_pred_final_nn_binary})

In [ ]:
new_df.to_csv('datathone_preds_2.0.csv',index=False)